# Вариант 8
## Молчанова

<div><img src="./Молчанова.png" width="450"/></div>

# Функция <em><strong>solve</strong></em> принимает следующие аргументы
##### k - список жесткостей пружин
##### c - список коэффициентов фемпфирования
##### m1,m2,m3 - массы тел
##### а1,а2 - расстояние от центра масс 2  тела до нижних пружин
##### а3 - расстояние от центра масс 2  тела до верхних пружин
##### h1,h4 - расстояния до верхних пружин от центра масс 1 тела по горизонтали
##### l1,l2,l3,l4,l5 - расстояния до нижних амортизаторов от центра масс 1 тела по горизонтали
##### t_matrix - список времен между удароми каждого амортизатора в период времени с (1,3) с шагом 0.002
##### I1,I2 - моменты инерции 

### А также имеет следующи константы:
##### delta_t=0.002
##### gamma = 0.5
##### betta = 0.25

In [ ]:
import numpy as np
#Функция для создания синусоиды
def sinys(t,amplisuda):
    return amplisuda*np.sin(np.pi*t)

def solve(k,c,m1,m2,m3,a1,a2,a3,h1,h4,l1,l2,l3,l4,l5,t_matrix,I1,I2,alfa,delta_t=0.002,gamma = 0.5,betta = 0.25):
    #Создаем матрицу коэффициентов упругости
    K_diag = np.diag(k)
    D = np.array([[0,0,-1/np.cos(alfa),a3/np.cos(alfa),1/np.cos(alfa)],
    [0,0,-1,0,1],
    [0,0,-1/np.cos(alfa),a3/np.cos(alfa),1/np.cos(alfa)],
    [-1,h1,1,-a1,0],
    [-1,-h4,1,a2,0],
    [1,-l1,0,0,0],
    [1,-l3,0,0,0],
    [1,l5,0,0,0],
    [1,l4,0,0,0],
    [1,l2,0,0,0]])
    K = np.transpose(D)@K_diag@D
    #Создаем матрицу коэффициентов демпфирования
    C_diag = np.diag(c)
    C = np.array([[0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0],
    [1,-l1,0,0,0],
    [1,-l3,0,0,0],
    [1,l5,0,0,0],
    [1,l4,0,0,0],
    [1,l2,0,0,0]])
    C = np.transpose(C)@C_diag@C
    #Создаем диагональную матрицу масс и моментов
    M = np.diag([m1,I1,m2,I2,m3])
    #Делаем матрицу внешних воздействий
    n = D.shape[0]-len(t_matrix)
    y_voz = np.zeros((D.shape[0]+1,1001))
    y_voz[0,:] = [i*0.001 for i in range(1000,3001,2)]
    index_t = [np.where(y_voz[0,:] == i)[-1][-1] for i in t_matrix]
    for i in index_t:
        y_voz[n,i:] =  np.concatenate([np.array([sinys(r,1) for r in np.linspace(0,1,75)]),np.zeros((1001-75-i))])
        n+=1
    y_impact = y_voz[1:,1:]
    #Создаем обратную матрицу
    M_reverse = np.linalg.inv(M+delta_t*gamma*C+delta_t**2*betta*K)
    #Создадим оператор для ввода вектора внешних сил
    K_diag_transpose = np.transpose(D)@K_diag
    #Зададим вектора перемещения скорости и ускорения в 0 момент
    X = np.zeros((K_diag_transpose.shape[0],1))
    X_t = np.zeros((K_diag_transpose.shape[0],1))
    X_tt = np.zeros((K_diag_transpose.shape[0],1))
    for i in range(y_impact.shape[1]-1):
        #Формируем матрицы методом Ньюмарка
        #Ускорений
        F = K_diag_transpose@y_impact[:,i+1]-C@(X_t[:,-1]+delta_t*(1-gamma)*X_tt[:,-1])-K@(X[:,-1]+delta_t*X_t[:,-1]+1/2*delta_t**2*(1-2*betta)*X_tt[:,-1])
        X_tt = np.concatenate([X_tt,(np.dot(F,M_reverse)).reshape(-1,1)],axis =1)
        #Скоростей
        X_tn = X_t[:,i]+delta_t*gamma**2*X_tt[:,-1]+(1-gamma)*X_tt[:,i]
        X_t = np.concatenate([X_t,X_tn.reshape(-1,1)],axis =1)
        #Перемещений
        X_n = X[:,i]+delta_t*X_t[:,i]+(1/2)*(delta_t**2)*2*betta*X_tt[:,-1]+(1-2*betta)*X_tt[:,i]
        X = np.concatenate([X,X_n.reshape(-1,1)],axis =1)
    #return X,X_t,X_tt
    #Если нужен вывод матриц ускорений,скоростей и перемещений нужно убрать # перед return